In [1]:
import base64
import collections
import datetime
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import string
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
from bokeh import io, plotting, models, layouts, palettes
import ipywidgets

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.0)
mpl.rcParams['figure.figsize'] = 10, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

plotting.output_notebook()

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/

Loading BokehJS ...

# Bar Charts

In [2]:
d = pd.DataFrame({
        'category': [(string.uppercase[i] * 5) for i in range(5)],
        'value': np.random.uniform(2, 5, 5)})

### Vertical Bar Chart

In [3]:
p = plotting.figure(
        plot_width=500,
        plot_height=500,
        title='This is a bar chart',
        tools='save',
        x_range=d.category.tolist())

p.vbar(
    x=range(1, len(d)+1),
    width=0.5,
    bottom=0,
    top=d.value,
    color='firebrick')

p.xaxis.axis_label = 'Category'
p.yaxis.axis_label = 'Value'

p.y_range = models.Range1d(0, 6)

plotting.show(p)

### Horizontal Bar Chart

In [4]:
p = plotting.figure(
        plot_width=700,
        plot_height=300,
        title='This is a bar chart',
        tools='save',
        y_range=d.category.tolist())

p.hbar(
    y=range(1, len(d)+1),
    height=0.5,
    left=0,
    right=d.value,
    color='firebrick')

p.yaxis.axis_label = 'Category'
p.xaxis.axis_label = 'Value'

p.x_range = models.Range1d(0, 6)

plotting.show(p)

### Stacked Bar Chart

In [5]:
# ?

### Grouped Bar Chart

In [6]:
# ?

# Scatterplots

In [7]:
d = pd.DataFrame({
        'x': np.random.randn(200),
        'y': np.random.randn(200),
        'size': np.random.uniform(4, 10, 200),
        'label': [(''.join(random.choice(string.lowercase) for i in range(5))) for i in range(200)],
        'color': [(random.choice(['red', 'blue', 'green'])) for i in range(200)]
    })

### Simple Scatterplot

In [8]:
p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a scatterplot',
        tools=['pan', 'wheel_zoom', 'reset', 'save'],
        active_drag='pan')

p.circle(
    x=d.x,
    y=d.y,
    size=5,
    color='firebrick',
    legend='data')

p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

plotting.show(p)

### Category Scatterplot

In [23]:
p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a scatterplot',
        tools=['pan', 'wheel_zoom', 'reset', 'save'],
        active_drag='pan')

for c in d.color.unique().tolist():
    p.circle(
        x=d[d.color==c].x,
        y=d[d.color==c].y,
        size=d[d.color==c]['size'],
        color=c,
        legend=c)

p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

p.xgrid.minor_grid_line_color = 'grey'
p.xgrid.minor_grid_line_alpha = 0.08
p.ygrid.minor_grid_line_color = 'grey'
p.ygrid.minor_grid_line_alpha = 0.08

plotting.show(p)

### Scatterplot with Annotations

In [10]:
source = models.ColumnDataSource(data=dict(
    x=d.x,
    y=d.y,
    color=d.color,
    size=d['size'],
    label=d.label,
))

labels = models.LabelSet(
    x='x',
    y='y',
    text='label',
    level='glyph',
    x_offset=5,
    y_offset=5,
    source=source,
    render_mode='canvas')

p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a scatterplot',
        tools=['pan', 'wheel_zoom', 'reset', 'save'],
        active_drag='pan')

p.add_layout(labels)

p.circle(
    x='x',
    y='y',
    size='size',
    color='color',
    legend='color',
    source=source)
    
p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

plotting.show(p)

### Scatterplot with Hover

In [21]:
source = models.ColumnDataSource(data=dict(
    x=d.x,
    y=d.y,
    color=d.color,
    size=d['size']*2,
    label=d.label,
))

hover = models.tools.HoverTool(tooltips=[
        ('label','@label'),
        ('x','@x'),
        ('y','@y'),
        ('color','@color'),
        ('size','@size')])

p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a scatterplot',
        tools=['pan', 'wheel_zoom', 'reset', 'save', hover],
        active_drag='pan')

p.circle(
    x='x',
    y='y',
    size='size',
    color='color',
    legend='color',
    source=source)
    
p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

plotting.show(p)

# Line Plots

In [54]:
d = pd.DataFrame({
        'x': np.linspace(1, 100, 100),
        'date_obj': pd.date_range(pd.datetime.today(), periods=100).tolist(),
        'date_str': [datetime.datetime.strftime(t, '%Y-%m-%d') for t in pd.date_range(pd.datetime.today(), periods=100).tolist()],
        'Foo': np.random.randn(100).cumsum(),
        'Bar': np.random.randn(100).cumsum(),
        'Baz': np.random.randn(100).cumsum()})

### Simple Line Plot

In [85]:
p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a lineplot',
        tools=['pan', 'wheel_zoom', 'reset', 'save'])

cat_to_col = {
    'Foo': 'red',
    'Bar': 'green',
    'Baz': 'blue'}

for cat in ['Foo', 'Bar', 'Baz']:
    p.line(
        x=d.x,
        y=d[cat],
        line_width=2,
        color=cat_to_col[cat],
        legend=cat)

p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

p.xgrid.minor_grid_line_color = 'grey'
p.xgrid.minor_grid_line_alpha = 0.08
p.ygrid.minor_grid_line_color = 'grey'
p.ygrid.minor_grid_line_alpha = 0.08

p.x_range = models.Range1d(1, 100)

plotting.show(p)

### Time Series Plot

In [78]:
p = plotting.figure(
        plot_width=900,
        plot_height=500,
        title='This is a time series plot',
        tools=['pan', 'wheel_zoom', 'reset', 'save'],
        x_axis_type='datetime')

cat_to_col = {
    'Foo': 'red',
    'Bar': 'green',
    'Baz': 'blue'}

for cat in ['Foo', 'Bar', 'Baz']:
    p.line(
        x=d.date_obj,
        y=d[cat],
        line_width=2,
        color=cat_to_col[cat],
        legend=cat)

p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

p.xgrid.minor_grid_line_color = 'grey'
p.xgrid.minor_grid_line_alpha = 0.08
p.ygrid.minor_grid_line_color = 'grey'
p.ygrid.minor_grid_line_alpha = 0.08

p.x_range = models.Range1d(d.date_obj.tolist()[0], d.date_obj.tolist()[-1])

plotting.show(p)

### Slope Graph

In [98]:
p = plotting.figure(
        plot_width=500,
        plot_height=500,
        title='This is a slope graph',
        tools=['save'])

cat_to_col = {
    'Foo': 'red',
    'Bar': 'green',
    'Baz': 'blue'}

for cat in ['Foo', 'Bar', 'Baz']:
    p.line(
        x=[1, 2],
        y=[d[cat].tolist()[0], d[cat].tolist()[1]],
        line_width=2,
        color=cat_to_col[cat])
    
for cat in ['Foo', 'Bar', 'Baz']:
    p.circle(
        x=[1, 2],
        y=[d[cat].tolist()[0], d[cat].tolist()[1]],
        size=5,
        color=cat_to_col[cat],
        legend=cat)

p.xaxis.axis_label = 'X axis'
p.yaxis.axis_label = 'Y axis'

p.x_range = models.Range1d(0.8, 2.2)

p.xaxis.ticker = [1, 2]
p.xaxis.axis_label = 'Before'
p.xgrid.visible = False
p.ygrid.minor_grid_line_color = 'grey'
p.ygrid.minor_grid_line_alpha = 0.08

p.legend.location = 'bottom_right'

plotting.show(p)